# Step 0.1. Import necessary libraries 

In [10]:
# Standard python libraries
import os
import time
import re

# Installed libraries
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

# Imports from our package
import lightgbm as lgb

# Step 0.2. Parameters 

In [11]:

TARGET_NAME = 'target'

# Step 0.3. Data load 

In [12]:
%%time

train_data = pd.read_csv('./input/train.csv')
train_data[TARGET_NAME] = train_data[TARGET_NAME].str.slice(start=6).astype(int) - 1
train_data.head()

CPU times: user 611 ms, sys: 65.7 ms, total: 676 ms
Wall time: 681 ms


,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,target
0,0,0,0,6,1,0,0,0,0,7,...,0,0,0,0,0,0,2,0,0,5
1,1,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,1,0,5
2,2,0,0,0,0,0,1,0,3,0,...,0,0,0,0,1,0,0,0,0,1
3,3,0,0,7,0,1,5,2,2,0,...,0,4,0,2,2,0,4,3,0,7
4,4,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [13]:
test_data = pd.read_csv('./input/test.csv')
test_data.head()

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74
0,200000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,200001,1,2,0,0,0,0,0,0,0,...,3,1,3,0,0,0,0,3,0,0
2,200002,0,1,7,1,0,0,0,0,6,...,3,0,0,0,0,3,0,2,0,0
3,200003,0,0,0,4,3,1,0,0,0,...,0,0,0,1,0,0,0,4,0,0
4,200004,0,0,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [14]:
submission = pd.read_csv('./input/sample_submission.csv')
submission.head()

,id,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
0,200000,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111
1,200001,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111
2,200002,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111
3,200003,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111
4,200004,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111


# Step 0.5. Add new features

In [15]:
def create_gr_feats(data):
    pass
    

all_df = pd.concat([train_data, test_data]).reset_index(drop = True)
create_gr_feats(all_df)
train_data, test_data = all_df[:len(train_data)], all_df[len(train_data):]
print(train_data.shape, test_data.shape)

(200000, 77) (100000, 77)


In [16]:
train_data.head()

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,target
0,0,0,0,6,1,0,0,0,0,7,...,0,0,0,0,0,0,2,0,0,5.0
1,1,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,1,0,5.0
2,2,0,0,0,0,0,1,0,3,0,...,0,0,0,0,1,0,0,0,0,1.0
3,3,0,0,7,0,1,5,2,2,0,...,0,4,0,2,2,0,4,3,0,7.0
4,4,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0


In [32]:
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss

In [41]:
remover = ['id', TARGET_NAME]

kf = KFold(n_splits=3, shuffle=True, random_state=0)

X = train_data.drop(remover, axis=1)
y = train_data[TARGET_NAME]


lls = []
for i, (tr, ts) in enumerate(kf.split(X)):
    Xtr, ytr = X.iloc[tr], y.iloc[tr]
    Xval, yval = X.iloc[ts], y.iloc[ts]
    
    #print(Xtr.shape, ytr.shape, Xval.shape, yval.shape)
    modelo = lgb.LGBMClassifier(n_estimators=100, 
                                learning_rate=0.05,
                                max_depth=6,
                                subsample=0.8,
                                colsample_bytree=0.8,
                                random_state=9, n_jobs=6)
    modelo.fit(Xtr, ytr)
    
    p = modelo.predict_proba(Xval)
    ll = log_loss(yval, p)
    
    lls.append(ll)
    print("Fold: {} | Log Loss: {:.4f}".format(i, ll))
    
print("Média da Loss: {:.4f}".format(np.mean(lls)))
    
    
# 1.7899
# 1.7531
# 1.7572
# 1.7540
# 1.7565

Fold: 0 | Log Loss: 1.7573
Fold: 1 | Log Loss: 1.7519
Fold: 2 | Log Loss: 1.7602
Média da Loss: 1.7565


## Step 4. Predict for test data and check OOF score

In [13]:
%%time

test_pred = automl.predict(test_data)
print('Prediction for test data:\n{}\nShape = {}'.format(test_pred[:10], test_pred.shape))

print('Check scores...')
print('OOF score: {}'.format(log_loss(train_data[TARGET_NAME].values, oof_pred.data)))

Prediction for test data:
array([[0.06476909, 0.3916093 , 0.15487702, 0.02635641, 0.01283768,
        0.15494484, 0.02237028, 0.04978392, 0.12245146],
       [0.04236097, 0.07071404, 0.05685499, 0.02075427, 0.01503311,
        0.2744168 , 0.08466147, 0.30401257, 0.13119176],
       [0.02001443, 0.02370084, 0.01564323, 0.00997829, 0.0063512 ,
        0.73568875, 0.02834253, 0.11780702, 0.04247368],
       [0.04652037, 0.11051005, 0.0837672 , 0.03297291, 0.01738988,
        0.25059748, 0.07831381, 0.22150673, 0.15842158],
       [0.04172738, 0.11030082, 0.08049527, 0.02490849, 0.01534015,
        0.28902256, 0.06611353, 0.22361517, 0.14847662],
       [0.04552011, 0.19149403, 0.10645209, 0.02604306, 0.01334615,
        0.29393595, 0.04544733, 0.1297297 , 0.14803158],
       [0.04401483, 0.11259232, 0.08304422, 0.03093427, 0.01896748,
        0.2223151 , 0.08736981, 0.24247345, 0.15828852],
       [0.04221279, 0.42807233, 0.19504562, 0.02217503, 0.01443897,
        0.03345913, 0.03714093,

## Step 5. Prepare submission

In [14]:
submission.iloc[:, 1:] = test_pred.data
submission.to_csv('lightautoml_2lvl_3hours.csv', index = False)

In [15]:
submission

,id,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
0,200000,0.064769,0.391609,0.154877,0.026356,0.012838,0.154945,0.022370,0.049784,0.122451
1,200001,0.042361,0.070714,0.056855,0.020754,0.015033,0.274417,0.084661,0.304013,0.131192
2,200002,0.020014,0.023701,0.015643,0.009978,0.006351,0.735689,0.028343,0.117807,0.042474
3,200003,0.046520,0.110510,0.083767,0.032973,0.017390,0.250597,0.078314,0.221507,0.158422
4,200004,0.041727,0.110301,0.080495,0.024908,0.015340,0.289023,0.066114,0.223615,0.148477
...,...,...,...,...,...,...,...,...,...,...
99995,299995,0.076573,0.373476,0.150132,0.030341,0.015206,0.102688,0.033028,0.075069,0.143487
99996,299996,0.050449,0.240075,0.129792,0.028192,0.014861,0.189044,0.050416,0.138321,0.158850
99997,299997,0.069616,0.285622,0.127926,0.029599,0.015611,0.172455,0.041709,0.104603,0.152860
99998,299998,0.033462,0.023512,0.020290,0.012150,0.010273,0.369281,0.074179,0.375746,0.081107
